In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
#階層クラスタリング
import fastcluster
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

In [2]:
df = pd.read_csv("../../data/horse_data_set.csv")
df["pred_race_date"] = pd.to_datetime(df["pred_race_date"],format='%Y-%m-%d')
df["past_race_date"] = pd.to_datetime(df["past_race_date"],format='%Y-%m-%d')

In [3]:
#sqlite 設定
con = sqlite3.connect('../../data/race_data_set.db') # データベースに接続する
con.enable_load_extension(True)
con.load_extension("../../../../sqlite-amalgamation-3260000/libsqlitefunctions.so")
cur = con.cursor()

In [5]:
sql_result = "select * from race_result"
sql_info   = "select * from race_info where distance <1700"
sql_pay = "select * from race_pay"

df_r = pd.read_sql_query(sql=sql_result,con=con)
df_i = pd.read_sql_query(sql=sql_info,con=con)

In [6]:
df_i.groupby(["distance","kind"]).count()

race_id  race_num  race_name  race_date  race_rank1  \
distance kind                                                        
1200     ダ        1299      1299       1299       1299        1299   
         芝        1000      1000       1000       1000        1000   
1300     ダ          99        99         99         99          99   
1400     ダ        1076      1076       1076       1076        1076   
         芝         503       503        503        503         503   
1500     芝          36        36         36         36          36   
1600     ダ         386       386        386        386         386   
         芝         906       906        906        906         906   

               race_rank2  condition  place  horse_count  lap_time  conner1  \
distance kind                                                                 
1200     ダ           1299       1299   1299         1299      1299        0   
         芝           1000       1000   1000         1000      1000        0   
1300     ダ             99         99     99           99        99        0   
1400     ダ           1076       1076   1076         1076      1076        0   
         芝            503        503    503          503       503        0   
1500     芝             36         36     36           36        36        0   
1600     ダ            386        386    386          386       386        0   
         芝            906        906    906          906       906        0   

               conner2  conner3  conner4  
distance kind                             
1200     ダ           0     1299     1299  
         芝           0     1000     1000  
1300     ダ           0       99       99  
1400     ダ           0     1076     1076  
         芝           0      503      503  
1500     芝          36       36       36  
1600     ダ           0      386      386  
         芝         307      906      906

In [145]:
#1200m
df_1200 = df_i[df_i["distance"]==1600]

In [146]:
d_train = {}
for i in df_1200[["race_id","lap_time"]].values:
    print(i)
    d_train[i[0]] = [float(i)/200 for i in i[1].split("-")]

[202005020202 '12.6-11.2-11.7-12.1-12.5-12.4-12.5-13.2']
[202005020204 '12.3-10.4-11.6-12.0-12.1-11.6-11.4-11.8']
[202005020206 '12.5-11.4-12.0-12.7-12.2-11.8-12.0-12.5']
[202005020212 '12.2-10.7-11.8-12.3-12.4-12.3-12.2-13.4']
[202008030211 '12.3-11.3-11.7-11.9-11.4-11.3-10.9-11.6']
[202005020101 '12.0-11.4-12.7-12.7-12.7-12.3-12.6-13.3']
[202005020108 '12.2-10.9-11.4-12.0-12.5-12.9-13.0-13.1']
[202005020111 '12.2-10.9-12.0-12.2-12.2-11.9-11.8-12.7']
[202008030104 '12.6-10.8-11.4-12.2-12.2-12.3-11.4-11.7']
[202008030110 '12.7-11.8-12.1-12.4-11.7-11.2-10.9-11.4']
[202006030808 '12.8-11.3-11.6-11.4-11.5-11.6-12.0-12.9']
[202006030812 '12.7-11.4-11.5-11.5-11.2-11.5-11.8-13.1']
[202009020804 '12.5-11.2-11.9-12.0-11.8-11.7-11.6-11.9']
[202006030705 '12.8-12.3-12.4-12.4-12.7-12.5-12.7-13.3']
[202009020711 '12.4-10.8-10.9-11.6-11.9-11.7-12.1-12.9']
[202006030605 '12.6-11.6-12.1-12.3-12.1-11.9-11.7-11.9']
[202006030608 '12.5-11.0-11.6-11.6-11.9-11.8-12.1-12.4']
[202009020607 '12.4-11.1-11.6-1

In [147]:
from sklearn.cluster import KMeans

n_clusters = 10
n_init = 10
max_iter = 300
tol = 0.0001
random_state = 1
n_jobs = 2

kmeans = KMeans(n_clusters=n_clusters,n_init=n_init,max_iter=max_iter,tol=tol,random_state=random_state,n_jobs=n_jobs)

#train/test
data = list(d_train.values())
train,test = train_test_split(np.array(data),train_size=0.8,shuffle=True)

In [148]:
kmeans.fit(train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=2, precompute_distances='auto',
       random_state=1, tol=0.0001, verbose=0)

In [149]:
kmeans.predict(test)

array([4, 2, 2, 9, 2, 7, 6, 7, 0, 4, 8, 1, 6, 8, 9, 8, 7, 1, 1, 9, 1, 4,
       5, 4, 6, 9, 9, 4, 3, 7, 0, 5, 8, 5, 1, 1, 2, 5, 6, 4, 5, 5, 6, 2,
       3, 7, 1, 3, 1, 8, 2, 9, 8, 1, 6, 6, 4, 5, 4, 2, 7, 3, 4, 4, 7, 1,
       1, 5, 1, 9, 0, 0, 0, 8, 2, 4, 2, 9, 8, 2, 2, 1, 8, 9, 2, 1, 7, 2,
       4, 2, 7, 6, 6, 2, 9, 7, 1, 6, 1, 9, 2, 1, 4, 2, 1, 0, 5, 6, 5, 1,
       7, 6, 2, 1, 1, 0, 2, 9, 6, 5, 9, 0, 1, 4, 6, 8, 5, 6, 0, 8, 6, 5,
       7, 9, 8, 7, 0, 2, 5, 9, 4, 6, 6, 0, 2, 9, 9, 2, 4, 4, 5, 9, 6, 4,
       1, 1, 6, 5, 5, 6, 0, 0, 4, 1, 6, 3, 1, 4, 1, 1, 4, 6, 2, 4, 7, 6,
       1, 5, 1, 3, 9, 9, 4, 8, 5, 6, 5, 6, 4, 5, 2, 1, 2, 9, 9, 5, 9, 8,
       8, 6, 4, 8, 3, 9, 2, 5, 5, 4, 7, 0, 4, 5, 2, 7, 4, 2, 0, 8, 9, 5,
       9, 8, 9, 0, 8, 8, 2, 2, 8, 6, 0, 2, 4, 1, 9, 2, 3, 5, 7, 1, 6, 6,
       1, 4, 4, 4, 7, 6, 4, 5, 8, 1, 6, 0, 5, 7, 0, 9, 3], dtype=int32)

In [151]:
Z = fastcluster.linkage_vector(train,method='ward',metric='euclidean')
Z_dataframe = pd.DataFrame(data=Z,columns = ['clus1','clus2','dis','newculssize'])

In [152]:
Z_dataframe

,clus1,clus2,dis,newculssize
0,502.0,820.0,0.000866,2.0
1,258.0,315.0,0.000866,2.0
2,629.0,837.0,0.001000,2.0
3,758.0,966.0,0.001000,2.0
4,622.0,630.0,0.001000,2.0
...,...,...,...,...
1027,2051.0,2052.0,0.044645,195.0
1028,2055.0,2057.0,0.064119,411.0
1029,2056.0,2060.0,0.073512,372.0
1030,2059.0,2061.0,0.095792,661.0


In [153]:
from scipy.cluster.hierarchy import fcluster
dis_thre = 0.02
cls = fcluster(Z,dis_thre,criterion="distance")
x_train_hier = pd.DataFrame(data=cls,columns=['aa'])

In [154]:
len(x_train_hier.aa.unique())

18

In [155]:
data = df_r[(df_r["popular"]==1) & (df_r["tyaku"]<4)]#& (df_r["odds"]>100)

In [156]:
data

,race_id,tyaku,wakuban,umaban,horse_id,horse_name,horse_sex,horse_years,jockey,kinryo,odds,popular,time,last3f,p1,p2,p3,p4,horse_weight,dif_horse_weight
17,202003010603,2,5,9,2017105356,ヴェルテックス,牡,3,川又賢治,55.0,1.7,1,109.2,39.8,3,3,2,2,526.0,2.0
32,202003010605,2,3,6,2017100381,オパールシャルム,牝,3,藤田菜七,52.0,2.4,1,71.1,35.8,0,0,8,7,472.0,2.0
47,202003010606,1,6,12,2017103000,バビット,牡,3,和田竜二,56.0,2.9,1,121.2,36.0,1,1,2,1,452.0,-4.0
63,202003010607,1,5,6,2016103729,モリノカワセミ,牝,4,木幡育也,53.0,2.5,1,108.9,38.6,8,8,7,5,458.0,-6.0
76,202003010609,1,7,8,2015106165,アトミカ,牝,5,西村淳也,54.0,2.6,1,109.5,35.1,4,4,6,6,444.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164279,201608010102,2,4,7,2013102492,ウインフェルベール,牡,3,川田将雅,56.0,4.4,1,73.5,37.7,0,0,2,2,476.0,4.0
164294,201608010103,1,7,12,2013106119,ゴールドドリーム,牡,3,川田将雅,56.0,2.5,1,112.3,36.9,5,5,5,3,524.0,-2.0
164308,201608010104,1,5,9,2012104572,オーロラポラリス,牡,4,太宰啓介,56.0,3.1,1,113.3,36.6,13,13,8,4,512.0,8.0
164325,201608010105,2,4,6,2013103564,アドマイヤキズナ,牡,3,岩田康誠,56.0,1.9,1,121.1,35.3,13,13,5,5,450.0,-8.0


In [157]:
len(df_i)

11415

In [168]:
from sklearn.decomposition import PCA
n_components = 8
whiten = False
random_state = 1

pca = PCA(n_components=n_components,whiten=whiten,random_state=random_state)

In [169]:
pca_t = pca.fit_transform(train)
pca_t = pd.DataFrame(data=pca_t)

In [170]:
pca_t

,0,1,2,3,4,5,6,7
0,-0.004516,-0.001922,0.000504,0.000809,0.000058,0.001221,0.000962,-0.000292
1,0.002016,0.006158,-0.003482,-0.001048,-0.002943,-0.000822,-0.000257,0.001049
2,-0.002648,-0.001313,0.000992,0.001813,0.000049,-0.000314,0.000439,-0.000192
3,0.003867,-0.000413,0.000300,-0.000325,0.000605,0.000450,0.003061,0.000464
4,0.001277,-0.006362,0.000257,-0.001622,-0.001715,0.001913,0.000200,0.000605
...,...,...,...,...,...,...,...,...
1028,0.006052,-0.000029,-0.001415,0.000363,0.001223,0.000732,0.000473,-0.000403
1029,-0.000489,-0.001703,-0.001232,-0.000764,0.000355,0.000265,0.000479,0.000220
1030,0.003902,-0.002197,0.000456,-0.001472,0.001611,-0.000720,0.000258,-0.001113
1031,-0.005585,0.002278,0.001453,0.000363,-0.000027,-0.000188,0.000829,-0.000886


作りたいmodel  
 - 凝集型階層クラスタリング  
 - オートエンコーダー  

In [92]:
#凝集型階層クラスタリング

#特徴量

#race_result
sql_r = """
            select *,
            round(time-top_time,2) as dif_time,
            round(time-lag_time,2) as dif_time1,
            round(lead_time-time,2) as dif_time2
            from (
            SELECT 
                rr.race_id as race_id, 
                rr.tyaku as tyaku,
                rr.wakuban as wakuban,
                rr.umaban as umaban,
                rr.horse_id,
                rr.horse_name,
                rr.horse_sex as horse_sex,
                rr.horse_years as horse_years,
                rr.kinryo as kinryo,
                rr.time,
                min(rr.time) over(partition by rr.race_id) as top_time,
                lag(rr.time,1) over(partition by rr.race_id order by rr.time) as lag_time, 
                lead(rr.time,1) over(partition by rr.race_id order by rr.time) as lead_time, 
                rr.last3f,
                rr.p1,
                rr.p2,
                rr.p3,
                rr.p4,
                rr.horse_weight as horse_weight,
                rr.dif_horse_weight as dif_horse_weight,
                ri.horse_count
                FROM race_result rr left join race_info ri on rr.race_id = ri.race_id )  tmp  
"""
#sql_mer = "select * from race_result rr left join race_info ri on rr.race_id = ri.race_id where ri.distance < 1700"
sql_info   = "select * from race_info where distance <1700"
df_i = pd.read_sql_query(sql=sql_info,con=con)
df_r = pd.read_sql_query(sql=sql_r,con=con)

In [93]:
#性別のカテゴライズ
cate_sex  = {'牝':0,'牡':1,'セ':2}
df_r['horse_sex'] = df_r['horse_sex'].apply(lambda x : cate_sex.get(x))
#位置取りの計算
df_r['dif_past1'] = df_r["p4"] - df_r["tyaku"]
df_r['dif_past2'] = df_r["tyaku"] / df_r["p4"]

df_r['p1'] = df_r["p1"] / df_r["horse_count"]
df_r['p2'] = df_r["p2"] / df_r["horse_count"]
df_r['p3'] = df_r["p3"] / df_r["horse_count"]
df_r['p4'] = df_r["p4"] / df_r["horse_count"]
df_r['pg'] = df_r["tyaku"] / df_r["horse_count"]

#競馬場のカテゴライズ
cate_place = {'東京':0,'中山':1,'阪神':2,'京都':3,'小倉':4,'新潟':5,'福島':6,'中京':7,'札幌':8,'函館':9}
df_i['place'] = df_i['place'].apply(lambda x : cate_place.get(x))
#馬場状態のカテゴライズ
cate_condition ={'良':0,'稍重':1,'稍':1,'重':2,'不良':3,'不':3}
df_i['condition'] = df_i['condition'].apply(lambda x : cate_condition.get(x))
#レースランク1のカテゴライズ
cate_rank1 = {'2歳':0,'3歳':1,'3歳以上':2,'4歳以上':3}
df_i['race_rank1'] = df_i['race_rank1'].apply(lambda x : cate_rank1.get(x))
#レースランク2のカテゴライズ
cate_rank2   = {'新馬':0,'未勝利':1,'500万下':2,'1000万下':3,'1600万下':4,'オープン':5,'1勝クラス':2,'2勝クラス':3,'3勝クラス':4,}
df_i['race_rank2'] = df_i['race_rank2'].apply(lambda x : cate_rank2.get(x))
#コースの種類のカテゴライズ
cate_kind = {'芝':0,'ダ':1}
df_i['kind'] = df_i['kind'].apply(lambda x : cate_kind.get(x))
#ラップタイムの変換
df_i["lapf1"] = None
df_i["lapf2"] = None
df_i["lapf3"] = None
df_i["lapf4"] = None
df_i["lapl3"] = None
df_i["lapl2"] = None
df_i["lapl1"] = None
df_i["dif_lap21"] = None
df_i["dif_lap32"] = None
df_i["dif_lap43"] = None
df_i["race_dif_3f"] = None
df_i["race_first_3f"] = None
df_i["race_last_3f"] = None
df_i[["lapf1","lapf2","lapf3","lapl3","lapf4","lapl2","lapl1"]] = [np.array(x.split('-'),dtype='float64')[[0,1,2,3,-3,-2,-1]] for x in df_i.lap_time]
df_i["lapf1"] = [x[0]/200.0 if (x[1]/100)%2 == 0 else x[0]/100.0 for x in df_i[["lapf1","distance"]].values]
df_i[["lapf2","lapf3","lapf4","lapl3","lapl2","lapl1"]] = df_i[["lapf2","lapf3","lapf4","lapl3","lapl2","lapl1"]]/200.0
df_i["race_first_3f"] = df_i["lapf1"]+ df_i["lapf2"] + df_i["lapf3"]
df_i["race_last_3f"]  = df_i["lapl1"]+ df_i["lapl2"] + df_i["lapl3"]
df_i["dif_lap21"] = df_i["lapf2"] - df_i["lapf1"]
df_i["dif_lap32"] = df_i["lapf3"] - df_i["lapf2"]
df_i["dif_lap43"] = df_i["lapf4"] - df_i["lapf3"]
df_i["race_dif_3f"] = df_i["race_first_3f"]-df_i["race_last_3f"]
df_i = df_i.drop(columns=["lapf4"])

In [102]:
df_id = df_r.set_index('race_id', drop=False)

In [353]:
df_i

,race_id,race_num,race_name,race_date,race_rank1,race_rank2,distance,kind,condition,place,...,lapf3,lapl3,lapl2,lapl1,dif_lap21,dif_lap32,dif_lap43,race_dif_3f,race_first_3f,race_last_3f
0,202003010602,2,3歳未勝利,2020-04-26,1,1,1200,0,0,6,...,0.0565,0.0585,0.0570,0.0625,-0.0080,0.0030,0.0020,-0.0065,0.1715,0.1780
1,202003010605,5,3歳未勝利,2020-04-26,1,1,1200,0,0,6,...,0.0560,0.0600,0.0595,0.0625,-0.0085,0.0015,0.0040,-0.0085,0.1735,0.1820
2,202003010612,12,米沢特別,2020-04-26,3,3,1200,0,0,6,...,0.0560,0.0590,0.0550,0.0590,-0.0090,0.0030,0.0030,-0.0020,0.1710,0.1730
3,202005020201,1,3歳未勝利,2020-04-26,1,1,1400,1,0,0,...,0.0615,0.0615,0.0630,0.0630,-0.0075,0.0070,0.0010,-0.0095,0.1780,0.1875
4,202005020202,2,3歳未勝利,2020-04-26,1,1,1600,1,0,0,...,0.0585,0.0605,0.0625,0.0660,-0.0070,0.0025,0.0035,-0.0115,0.1775,0.1890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5300,201606010110,10,ジュニアカップ,2016-01-05,1,5,1600,0,0,1,...,0.0590,0.0595,0.0565,0.0585,-0.0070,0.0015,-0.0020,0.0065,0.1810,0.1745
5301,201606010112,12,4歳以上1000万下,2016-01-05,3,3,1600,0,0,1,...,0.0550,0.0585,0.0570,0.0575,-0.0075,-0.0015,0.0030,0.0025,0.1755,0.1730
5302,201608010102,2,3歳未勝利,2016-01-05,1,1,1200,1,0,3,...,0.0595,0.0615,0.0620,0.0655,-0.0075,0.0040,0.0020,-0.0110,0.1780,0.1890
5303,201608010107,7,4歳以上500万下,2016-01-05,3,2,1200,1,0,3,...,0.0585,0.0590,0.0610,0.0630,-0.0065,0.0020,0.0005,-0.0050,0.1780,0.1830


In [268]:
df_222 = df_r[(df_r["race_id"]==201601010204) | (df_r["race_id"]==202010011203)]
df_223 = df_i[(df_i["race_id"]==201601010204) | (df_i["race_id"]==202010011203)].set_index("race_id", drop=False)

In [342]:
df_r

,race_id,tyaku,wakuban,umaban,horse_id,horse_name,horse_sex,horse_years,kinryo,time,...,p4,horse_weight,dif_horse_weight,horse_count,dif_time,dif_time1,dif_time2,dif_past1,dif_past2,pg
0,201601010103,1,6,6,2013102336,トウカイハッピー,1,3,56.0,154.9,...,0.454545,490.0,0.0,11,0.0,NaN,0.1,4,0.200000,0.090909
1,201601010103,2,6,7,2013103068,ウィーバービル,1,3,56.0,155.0,...,0.181818,510.0,0.0,11,0.1,0.1,0.3,0,1.000000,0.181818
2,201601010103,3,8,11,2013103490,メイショウルルド,1,3,56.0,155.3,...,0.090909,512.0,2.0,11,0.4,0.3,0.5,-2,3.000000,0.272727
3,201601010103,4,5,5,2013101031,シーザワールド,1,3,56.0,155.8,...,0.272727,496.0,-10.0,11,0.9,0.5,0.4,-1,1.333333,0.363636
4,201601010103,5,8,10,2013103498,ウインウェルス,1,3,56.0,156.2,...,0.545455,506.0,12.0,11,1.3,0.4,0.0,1,0.833333,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164417,202010011212,8,6,8,2016106315,ギルマ,1,4,53.0,163.9,...,0.333333,488.0,-8.0,12,0.8,0.3,0.7,-4,2.000000,0.666667
164418,202010011212,9,5,5,2014106464,テーオーフォルテ,1,6,57.5,164.6,...,0.500000,516.0,2.0,12,1.5,0.7,0.4,-3,1.500000,0.750000
164419,202010011212,10,3,3,2015104367,ジャディード,1,5,54.0,165.0,...,0.833333,448.0,4.0,12,1.9,0.4,2.6,0,1.000000,0.833333
164420,202010011212,11,1,1,2014103084,エスティーム,1,6,55.0,167.6,...,0.916667,524.0,0.0,12,4.5,2.6,0.4,0,1.000000,0.916667


In [354]:
%%time
data=[]
index_d = df_r.columns
for name,group in df_r.groupby("race_id"):#.apply(lambda x: x.sort_values(by="umaban"))
    ddd= []
    for d in group.sort_values(by="umaban").set_index("race_id",drop=False).values:
        #print(d)
        sss = ["{}_{}".format(col,d[3]) for col in (index_d)]
        ddd.append(pd.Series(d,name=name,index=sss))
    fff= pd.concat(ddd)
    data.append(pd.DataFrame(fff).T)
data = pd.concat(data)
data = df_i.join(data)

CPU times: user 2min 47s, sys: 71.1 ms, total: 2min 47s
Wall time: 2min 47s


In [349]:
dfdfd=df_i.set_index("race_id")#.join(data ,left_index=True, right_index=True)

In [352]:
dfdfd.join(data)

,race_num,race_name,race_date,race_rank1,race_rank2,distance,kind,condition,place,horse_count,...,p4_18,horse_weight_18,dif_horse_weight_18,horse_count_18,dif_time_18,dif_time1_18,dif_time2_18,dif_past1_18,dif_past2_18,pg_18
race_id,,,,,,,,,,,,,,,,,,,,,
202003010602,2,3歳未勝利,2020-04-26,1,1,1200,0,0,6,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202003010605,5,3歳未勝利,2020-04-26,1,1,1200,0,0,6,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202003010612,12,米沢特別,2020-04-26,3,3,1200,0,0,6,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202005020201,1,3歳未勝利,2020-04-26,1,1,1400,1,0,0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202005020202,2,3歳未勝利,2020-04-26,1,1,1600,1,0,0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201606010110,10,ジュニアカップ,2016-01-05,1,5,1600,0,0,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201606010112,12,4歳以上1000万下,2016-01-05,3,3,1600,0,0,1,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201608010102,2,3歳未勝利,2016-01-05,1,1,1200,1,0,3,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
data

,race_id_1,tyaku_1,wakuban_1,umaban_1,horse_id_1,horse_name_1,horse_sex_1,horse_years_1,kinryo_1,time_1,...,p4_18,horse_weight_18,dif_horse_weight_18,horse_count_18,dif_time_18,dif_time1_18,dif_time2_18,dif_past1_18,dif_past2_18,pg_18
201601010103,201601010103,7,1,1,2013105247,ストラーノ,1,3,53,156.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201601010104,201601010104,6,1,1,2013103042,アグネスルコリエ,0,3,51,107.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201601010106,201601010106,2,1,1,2013103153,ギャツビー,1,3,53,123.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201601010108,201601010108,3,1,1,2013103430,フジマサアクトレス,0,3,52,69.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201601010109,201601010109,10,1,1,2011101936,スリーチアサウス,0,5,53,106.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202010011208,202010011208,11,1,1,2016104682,フランクリン,1,4,55,124.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202010011209,202010011209,5,1,1,2016103091,クラウドスケープ,1,4,56,105.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202010011210,202010011210,17,1,1,2017100601,ヒバリ,0,3,54,72.7,...,0.833333,458,4,18,1.2,0.1,0.1,5,0.666667,0.555556
202010011211,202010011211,8,1,1,2015104921,アロハリリー,0,5,54,109.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
fff = pd.DataFrame(fff)

In [293]:
type(fff.T)

pandas.core.frame.DataFrame

In [294]:
fff.T

,race_id,tyaku,wakuban,umaban,horse_id,horse_name,horse_sex,horse_years,kinryo,time,...,p4,horse_weight,dif_horse_weight,horse_count,dif_time,dif_time1,dif_time2,dif_past1,dif_past2,pg
0,201601010204,1,1,1,2013100640,ドルチャーリオ,1,3,56,89.4,...,0.777778,440,4,18,3.1,0.3,NaN,-4,1.28571,1


In [333]:

#1億個の数字を足す関数

result = 0
for no in range(100000000):
    result += no

CPU times: user 8.27 s, sys: 0 ns, total: 8.27 s
Wall time: 8.27 s


In [244]:
for i in fff.T.columns:
    print(i)

race_id
tyaku
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
tyaku
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
tyaku
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
tyaku
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
tyaku
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lea

In [246]:
for i in fff.T.drop("tyaku",axis=1).columns:
    print(i)

race_id
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
horse_weight
dif_horse_weight
horse_count
dif_time
dif_time1
dif_time2
dif_past1
dif_past2
pg
race_id
wakuban
umaban
horse_id
horse_name
horse_sex
horse_years
kinryo
time
top_time
lag_time
lead_time
last3f
p1
p2
p3
p4
hors

In [114]:
result

,A,B,C
K0,a,0,A
K1,b,1,B
K2,c,2,C


In [23]:
df.columns
'race_id', 'umaban', 'horse_id', 'horse_name',
       'horse_sex', 'horse_years', 'jockey', 'kinryo', 'popular',
       'time', 'last3f', 'p1', 'p2', 'p3', 'p4', 'horse_weight',
       'dif_horse_weight', 'race_id', 'race_num', 'race_name', 'race_date',
       'race_rank1', 'race_rank2', 'distance', 'kind', 'condition', 'place',
       'horse_count', 'lap_time', 'conner1', 'conner2', 'conner3', 'conner4',
       'dif_past1', 'dif_past2', 'pg', 'lapf1', 'lapf2', 'lapf3', 'lapl3',
       'lapl2', 'lapl1', 'dif_lap21', 'dif_lap32', 'dif_lap43', 'race_dif_3f',
       'race_first_3f', 'race_last_3f'

Index(['race_id', 'tyaku', 'wakuban', 'umaban', 'horse_id', 'horse_name',
       'horse_sex', 'horse_years', 'jockey', 'kinryo', 'odds', 'popular',
       'time', 'last3f', 'p1', 'p2', 'p3', 'p4', 'horse_weight',
       'dif_horse_weight', 'race_id', 'race_num', 'race_name', 'race_date',
       'race_rank1', 'race_rank2', 'distance', 'kind', 'condition', 'place',
       'horse_count', 'lap_time', 'conner1', 'conner2', 'conner3', 'conner4',
       'dif_past1', 'dif_past2', 'pg', 'lapf1', 'lapf2', 'lapf3', 'lapl3',
       'lapl2', 'lapl1', 'dif_lap21', 'dif_lap32', 'dif_lap43', 'race_dif_3f',
       'race_first_3f', 'race_last_3f'],
      dtype='object')

In [30]:
df1.columns

Index(['race_id', 'tyaku', 'wakuban', 'umaban', 'horse_id', 'horse_name',
       'horse_sex', 'horse_years', 'kinryo', 'time', 'top_time', 'lag_time',
       'lead_time', 'last3f', 'p1', 'p2', 'p3', 'p4', 'horse_weight',
       'dif_horse_weight', 'race_date', 'race_rank1', 'race_rank2', 'distance',
       'kind', 'condition', 'place', 'horse_count', 'lap_time', 'dif_time',
       'dif_time1', 'dif_time2', 'dif_past1', 'dif_past2', 'pg', 'lapf1',
       'lapf2', 'lapf3', 'lapl3', 'lapl2', 'lapl1', 'dif_lap21', 'dif_lap32',
       'dif_lap43', 'race_dif_3f', 'race_first_3f', 'race_last_3f'],
      dtype='object')

In [31]:
df.head()

,race_id,tyaku,wakuban,umaban,horse_id,horse_name,horse_sex,horse_years,kinryo,time,...,lapf3,lapl3,lapl2,lapl1,dif_lap21,dif_lap32,dif_lap43,race_dif_3f,race_first_3f,race_last_3f
0,201601010103,1,6,6,2013102336,トウカイハッピー,1,3,56.0,154.9,...,0.062,0.0615,0.064,0.0665,-0.005,0.0,0.0025,-0.001,0.191,0.192
1,201601010103,2,6,7,2013103068,ウィーバービル,1,3,56.0,155.0,...,0.062,0.0615,0.064,0.0665,-0.005,0.0,0.0025,-0.001,0.191,0.192
2,201601010103,3,8,11,2013103490,メイショウルルド,1,3,56.0,155.3,...,0.062,0.0615,0.064,0.0665,-0.005,0.0,0.0025,-0.001,0.191,0.192
3,201601010103,4,5,5,2013101031,シーザワールド,1,3,56.0,155.8,...,0.062,0.0615,0.064,0.0665,-0.005,0.0,0.0025,-0.001,0.191,0.192
4,201601010103,5,8,10,2013103498,ウインウェルス,1,3,56.0,156.2,...,0.062,0.0615,0.064,0.0665,-0.005,0.0,0.0025,-0.001,0.191,0.192
